In [51]:
import re 
#input= 'a?a((cd)|(a|b))b+bb'
input = 'a(a|b)*b'
removed_zero_or_one_symbol = re.sub(r'(\w)\?', r'(\1|)', input)
removed_one_or_more_symbol = re.sub(r'(\w)\+', r'\1\1*', removed_zero_or_one_symbol)
#TODO : store $  in the dict and replace with the value
#TODO : Add $ to the alpha numeric in the forloop
#TODO : question : ask about the dot operator .
removed_match_brackets = re.sub(r'\[.*\]', r'$', removed_one_or_more_symbol) # for now one match 
print(removed_match_brackets) 
removed_match_brackets = removed_match_brackets + '^'
i = 0
while (1):
    if ( removed_match_brackets [i] == '^' ) : break
    if  ( removed_match_brackets[i+1].isalnum() or removed_match_brackets[i+1] == '(')  and ( removed_match_brackets[i].isalnum() or removed_match_brackets[i] == ')' ) :
        removed_match_brackets = removed_match_brackets[:i+1] + '?' + removed_match_brackets[i+1:]
    i = i + 1
print (removed_match_brackets)
input = removed_match_brackets


a(a|b)*b
a?(a|b)*b^


In [ ]:

precedence_dict = {'*': 3, '?': 2, '|': 1}
out =[]
operator_stack = []
input = 'a?(a|b)*?b^'
for char in input :
    if char.isalnum() :
        out.append(char)
    elif  char in precedence_dict.keys() :
        if len(operator_stack) == 0:
            operator_stack.append(char)
        elif operator_stack[-1] =='('or precedence_dict[ operator_stack[-1] ] < precedence_dict[char] :
            operator_stack.append(char)
        elif operator_stack[-1] != '(' and precedence_dict[ operator_stack[-1] ] == precedence_dict[char] : 
            out.append(char)
        else :
            while (len(operator_stack)>0 and operator_stack[-1] != '('and precedence_dict[ operator_stack[-1] ]  >= precedence_dict[char]) :
                popped_operator = operator_stack.pop()
                out.append(popped_operator)
            operator_stack.append(char) 
    elif char == '(' :
        operator_stack.append(char)

    elif char == ')' :
       
        while operator_stack:
            operator = operator_stack.pop()
            if operator == '(':
                break
            out.append(operator)
    print ("parsing char :",char,"output:",out,"stack",operator_stack)
    print("************************")


parsing char : a output: ['a'] stack []
************************
parsing char : ? output: ['a'] stack ['?']
************************
parsing char : ( output: ['a'] stack ['?', '(']
************************
parsing char : a output: ['a', 'a'] stack ['?', '(']
************************
parsing char : | output: ['a', 'a'] stack ['?', '(', '|']
************************
parsing char : b output: ['a', 'a', 'b'] stack ['?', '(', '|']
************************
parsing char : ) output: ['a', 'a', 'b', '|'] stack ['?']
************************
parsing char : * output: ['a', 'a', 'b', '|'] stack ['?', '*']
************************
parsing char : ? output: ['a', 'a', 'b', '|', '*', '?'] stack ['?']
************************
parsing char : b output: ['a', 'a', 'b', '|', '*', '?', 'b'] stack ['?']
************************
parsing char : ^ output: ['a', 'a', 'b', '|', '*', '?', 'b'] stack ['?']
************************
